In [1]:
import sys
if sys.version_info[0] < 3:
    # python 2 backwards compatibility
    import Queue as queue
else:
    import queue
import threading
import os
import logging

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Sequence
from sqlalchemy import Column, Integer, Float, Boolean, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm.scoping import scoped_session

In [2]:
import time as time_module
class Time(object):
    def __init__(self, timelapse = 1.0):
        self.timelapse = timelapse
        self.start_time = time_module.time()
    def time(self):
        return (time_module.time() - self.start_time) * self.timelapse + self.start_time
    def sleep(self, sleeptime):
        return time_module.sleep(sleeptime)

In [3]:
overwrite_db = True
if overwrite_db:
    os.remove('test.db')
engine = create_engine('sqlite:///test.db', echo=False)
Base = declarative_base()
Base.metadata.create_all(engine)
sessionfactory = sessionmaker(bind=engine)

In [4]:
class TemperatureSensor1(Base):
    __tablename__ = 'temperature sensor 1'
    id = Column(Integer, Sequence('TemperatureSensor1_id_seq'), primary_key=True)
    time = Column(Float) # seconds since the Epoch
    temperature = Column(Float) # centigrade

    def __repr__(self):
        return "<TemperatureSensor1(time='%f', temperature='%f')>" % (self.time, self.temperature) 

def Read_TemperatureSensor1():
    session = scoped_session(sessionfactory)
    while True:
        T=-999.999 # get temperature in centigrade
        update = TemperatureSensor1(time=time.time(), temperature=T)
        session.add(update)
        session.commit()
        time.sleep(1)
        #now = session.query(TemperatureSensor1).order_by(TemperatureSensor1.id.desc()).first()
        #now
        #now.time
        #now.temperature
        #for instance in session.query(TemperatureSensor1).order_by(TemperatureSensor1.id.desc()):
            #print instance.time, instance.temperature  

In [5]:
class Heater1(Base):
    __tablename__ = 'heater 1'
    id = Column(Integer, Sequence('Heater1_id_seq'), primary_key=True)
    time = Column(Float) # seconds since the Epoch
    on_state = Column(Boolean) # True is on, False is off

    def __repr__(self):
        return "<Heater1(time='%f', on_state='%s')>" %(self.time, str(self.on_state)) 

def Read_Heater1():
    session = scoped_session(sessionfactory)
    while True:
        on = True # get temperature in centigrade
        update = Heater1(time= time.time(), on_state = on)
        session.add(update)
        session.commit()
        time.sleep(1)
        #now = session.query(Heater1).order_by(Heater1.id.desc()).first()
        #now
        #now.time
        #now.temperature
        #for instance in session.query(Heater1).order_by(Heater1.id.desc()):
            #print instance.time, instance.temperature  

In [6]:
Base.metadata.create_all(engine)

# time = Time() # realtime mode
time = Time(timelapse = 2.0) # testing timelapse mode

q = queue.Queue()
t = threading.Thread(target=Read_TemperatureSensor1)
t.daemon = True
t.start()
time.sleep(.1) # give threads time to populate database

In [7]:
session = sessionfactory()
def get_last(session, table, timeout=2.):
    value = session.query(table).order_by(table.id.desc()).first()
    if time.time() > value.time + timeout:
        logging.warning('last value older than %3.2fs: %s' % (timeout, table))
    return value

In [18]:
print(get_last(session, TemperatureSensor1))

<TemperatureSensor1(time='1444629786.711926', temperature='-999.999000')>
